## Readability

This one takes in readability into account, finds the Flesch reading ease and gunning fog reading and saves it to a column.

This blog also finds grammar mistakes via the Language Tool library and saves them to a column also

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType, IntegerType
import language_tool_python as ltp
import textstat

# Set up Spark
conf = SparkConf().setAppName("Article Clarity Metrics")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# Read in Parquet file containing article text
df = spark.read.parquet("path/to/articles.parquet")

# Define functions to calculate readability scores and grammar mistakes
flesch_reading_ease = udf(lambda text: textstat.flesch_reading_ease(text), FloatType())
gunning_fog = udf(lambda text: textstat.gunning_fog(text), FloatType())
tool = ltp.LanguageTool('en-US')
grammar_mistakes = udf(lambda text: len(tool.check(text)), IntegerType())

# Add columns for readability scores and grammar mistakes
df = df.withColumn("flesch_reading_ease", flesch_reading_ease("text"))
df = df.withColumn("gunning_fog_index", gunning_fog("text"))
df = df.withColumn("grammar_mistakes", grammar_mistakes("text"))

# Save results as Parquet file
df.write.parquet("path/to/results.parquet")

This one below is from Bard:

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame

def readability(df):
  # Compute the Flesch reading ease score for each article.
  df["flesch_reading_ease"] = df["words"].toDouble() / df["sentences"].toDouble()

  # Compute the Gunning fog index for each article.
  df["gunning_fog_index"] = 2 * df["words"].toDouble() / df["sentences"].toDouble() + 12

  # Rank the articles by their readability score.
  df.sortBy(df["flesch_reading_ease"], ascending=False).show()

def main(args):
  # Create a SparkSession.
  spark = SparkSession.builder().master("local").appName("readability").build()

  # Create a DataFrame of words.
  words = spark.createDataFrame(
    [
      "The", "quick", "brown", "fox", "jumped", "over", "the", "lazy", "dog"
    ]
  )

  # Compute the Flesch reading ease score for each article.
  words.withColumn("sentences", words.words.split(" ").count()).withColumn("flesch_reading_ease", words.words.toDouble() / words.sentences.toDouble())

  # Compute the Gunning fog index for each article.
  words.withColumn("gunning_fog_index", 2 * words.words.toDouble() / words.sentences.toDouble() + 12)

  # Rank the articles by their readability score.
  words.sortBy(words["flesch_reading_ease"], ascending=False).show()

if __name__ == "__main__":
  main(args)

Further ones to check out that may be useful:

 - DependencyParserModel - analyzes grammatical structure
 - ChunkerModel - groups related words in a sentence
 - SentenceDetectorDLModel - sentence boundary detection to split text into individual sentences, to prepare data for named entity recognition and sentiment analysis
 - NERDLModel - identify named entities
 - SentimentDLModel - sentiment analysis for sentences positive, negative or neutral
 - MultiClassifierDLModel - multi-class specification to classify text data into multiple categories, classifies text data into topics, genres, or other sets. 